#Import libraries and load data

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Model
import numpy as np
print(tf.__version__)

2.2.0-rc2


In [2]:
mnist = tf.keras.datasets.fashion_mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()
training_images=training_images.reshape(60000, 28, 28, 1)
training_images=training_images / 255.0
test_images = test_images.reshape(10000, 28, 28, 1)
test_images=test_images/255.0

4423680/4422102 [==============================] - 0s 0us/step


##Make the model with functional api


In [0]:
input_img = tf.keras.layers.Input(shape=(28,28,1))
encoded = tf.keras.layers.Conv2D(64, (3,3), activation='relu')(input_img)
encoded = tf.keras.layers.MaxPooling2D(2, 2)(encoded)
encoded = tf.keras.layers.Conv2D(64, (3,3), activation='relu')(encoded)
encoded = tf.keras.layers.MaxPooling2D(2,2)(encoded)
encoded = tf.keras.layers.Flatten()(encoded)
encoded = tf.keras.layers.Dense(128, activation='relu')(encoded)
decoded = tf.keras.layers.Dense(10, activation='softmax')(encoded)

model = Model(input_img, decoded)
encoder = Model(input_img, encoded)

In [4]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 26, 26, 64)        640       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               204928

##Train and evaluate the model

In [5]:
model.fit(training_images, training_labels, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.4393 - accuracy: 0.8407
Epoch 2/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2931 - accuracy: 0.8928
Epoch 3/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2502 - accuracy: 0.9066
Epoch 4/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2153 - accuracy: 0.9192
Epoch 5/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1885 - accuracy: 0.9283


In [6]:
test_loss = model.evaluate(test_images, test_labels)

313/313 [==============================] - 1s 3ms/step - loss: 0.2408 - accuracy: 0.9148


It's likely gone up to about 93% on the training data and 91% on the validation data. 


## cosine similarity of the last layer


In [0]:
def cos_sim(x, y):
    return np.dot(x, y) / (np.linalg.norm(x, 2) * np.linalg.norm(y, 2)) 

In [8]:
print(test_labels[:30])
print(np.argmax(model.predict(test_images[:30]), axis=1))

[9 2 1 1 6 1 4 6 5 7 4 5 7 3 4 1 2 4 8 0 2 5 7 9 1 4 6 0 9 3]
[9 2 1 1 6 1 4 6 5 7 4 5 7 3 4 1 2 4 8 0 2 5 7 5 1 4 6 0 9 3]


In [11]:
x1 = np.expand_dims(test_images[3], axis=0) #select index 
x2 = np.expand_dims(test_images[2], axis=0) #select index

f1 = encoder.predict(x1)
f2 = encoder.predict(x2)

print(cos_sim(f1[0], f2[0]))

0.9526834
